In [1]:
import optuna
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
import torch.optim as optim
from torch import nn
import torch
import time

from GCN_network import GCNNetwork,train,test,validation

c:\Users\User\anaconda3\envs\cnn-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset     = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
data        = dataset[0]
FEATURES    = dataset.num_features
CLASSES     = dataset.num_classes
EDGE_INDEX  = data.edge_index

EPOCHS      = 100
EPOCHS_RETRAIN = 10

In [3]:
def objective(trial):
    """Funcion objetivo para optimizar la red neuronal
    
    Args:
    trial (optuna.trial): Objeto de la clase trial de optuna
    
    Returns:
    float: test_accuracy obtenido en la validacion"""

    #se define el numero de capaz
    n_layers                = trial.suggest_int(f'n_layers', 1, 3)
    channels_per_layer      = []

    #se calcula el numero de neuronas por capa
    for i in range(n_layers):
        channels  = trial.suggest_int(f'n_chanels_l{i}', 50, 500)
        channels_per_layer.append(channels)

    #se instancia el modelo, con los diferentes parametros
    model               = GCNNetwork(FEATURES, CLASSES, channels_per_layer)

    #se define la funcion de perdida y el optimizador
    optimizer_name      = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr                  = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer           = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    loss_fn             = nn.CrossEntropyLoss()

    #se realiza el entrenamiento
    for epoch in range(1, EPOCHS + 1):
        train_loss     = train(data,model,loss_fn, optimizer,EDGE_INDEX)
        val_acc        = validation(data, model,EDGE_INDEX)
        #se reporta el 'accuracy obtenido luego de la validacion'
        trial.report(val_acc, epoch)

        #si el entrenamiento no es prometedor, se detiene el entrenamiento
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    return val_acc

In [4]:
#se crea el estudio de optuna
study_name      = "GCN-training"  

#se carga el estudio si ya existe y se dirige la direccion hacia donde optuna realizara la optimizacion
study           = optuna.create_study(direction='maximize',load_if_exists=True,study_name=study_name)
study.optimize(objective, n_trials=20)

[I 2024-10-23 13:00:29,746] A new study created in memory with name: GCN-training
[I 2024-10-23 13:00:54,306] Trial 0 finished with value: 12.8 and parameters: {'n_layers': 2, 'n_chanels_l0': 485, 'n_chanels_l1': 82, 'optimizer': 'SGD', 'lr': 0.00016950818420477352}. Best is trial 0 with value: 12.8.
[I 2024-10-23 13:01:00,956] Trial 1 finished with value: 78.8 and parameters: {'n_layers': 2, 'n_chanels_l0': 62, 'n_chanels_l1': 234, 'optimizer': 'RMSprop', 'lr': 0.0038146836237282562}. Best is trial 1 with value: 78.8.
[I 2024-10-23 13:01:05,996] Trial 2 finished with value: 27.400000000000002 and parameters: {'n_layers': 1, 'n_chanels_l0': 100, 'optimizer': 'Adam', 'lr': 2.1128158057259008e-05}. Best is trial 1 with value: 78.8.
[I 2024-10-23 13:01:31,838] Trial 3 finished with value: 24.0 and parameters: {'n_layers': 3, 'n_chanels_l0': 313, 'n_chanels_l1': 491, 'n_chanels_l2': 92, 'optimizer': 'SGD', 'lr': 0.02018428087742989}. Best is trial 1 with value: 78.8.
[I 2024-10-23 13:01:52

In [5]:
#se imprimen los mejores hiperparmametros del estudio
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  78.8
  Params: 
    n_layers: 2
    n_chanels_l0: 62
    n_chanels_l1: 234
    optimizer: RMSprop
    lr: 0.0038146836237282562


In [6]:
#se obtienen los mejores parametros y se re entrena el modelo con estos parametros
n_layers                = study.best_trial.params["n_layers"]
channels_per_layer      = []

for i in range(n_layers ):
    channels_per_layer.append(study.best_trial.params[f"n_chanels_l{i}"])
    
model               = GCNNetwork(FEATURES, CLASSES, channels_per_layer)
optimizer_name   = study.best_trial.params["optimizer"]
lr               = study.best_trial.params["lr"]
optimizer        = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
loss_fn          = nn.CrossEntropyLoss()

#se realiza el re-entrenamiento
for epoch in range(1, EPOCHS_RETRAIN + 1):
    train_loss         = train(data,model,loss_fn, optimizer,EDGE_INDEX)
    val_acc            = validation(data, model,EDGE_INDEX)
    print(f"Epoch {epoch}/{EPOCHS_RETRAIN} - val acc: {val_acc:.4f}")

Epoch 1/10 - val acc: 5.8000
Epoch 2/10 - val acc: 13.6000
Epoch 3/10 - val acc: 19.8000
Epoch 4/10 - val acc: 38.2000
Epoch 5/10 - val acc: 62.2000
Epoch 6/10 - val acc: 22.2000
Epoch 7/10 - val acc: 51.6000
Epoch 8/10 - val acc: 67.4000
Epoch 9/10 - val acc: 72.8000
Epoch 10/10 - val acc: 72.0000


In [7]:
#se evalua el rendimiento final del modelo
test_acc = test(data, model,EDGE_INDEX)
print(f"Test accuracy: {test_acc:.4f}")

Test accuracy: 77.0000
